In [1]:
import os 
import cv2 
import numpy as np 
import tensorflow as tf 
import sys

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.98)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [3]:
sys.path.append("..") 


In [4]:
from object_detection.utils import label_map_util 
from object_detection.utils import visualization_utils as vis_util


In [5]:
IMAGE_NAME = 'ambulance125.jpeg'

In [6]:
CWD_PATH = os.getcwd() 


In [7]:
PATH_TO_CKPT = 'frozen_inference_graph.pb'


In [8]:
PATH_TO_LABELS = 'annotations/label_map.pbtxt'


In [9]:
PATH_TO_IMAGE =  'ambulance125.jpeg'

In [10]:
NUM_CLASSES = 1


In [11]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS) 
categories = label_map_util.convert_label_map_to_categories( 
        label_map, max_num_classes = NUM_CLASSES, use_display_name = True) 
category_index = label_map_util.create_category_index(categories)

In [12]:
detection_graph = tf.Graph() 
with detection_graph.as_default(): 
    od_graph_def = tf.compat.v1.GraphDef() 
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
        serialized_graph = fid.read() 
        od_graph_def.ParseFromString(serialized_graph) 
        tf.import_graph_def(od_graph_def, name ='') 
  
    sess = tf.compat.v1.Session(graph = detection_graph) #v2 

In [13]:
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 


In [14]:
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 


In [15]:
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 

In [16]:
num_detections = detection_graph.get_tensor_by_name('num_detections:0') 


In [17]:
image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis = 0) 
  
# Perform the actual detection by running the model with the image as input 
(boxes, scores, classes, num) = sess.run( 
    [detection_boxes, detection_scores, detection_classes, num_detections], 
    feed_dict ={image_tensor: image_expanded}) 
  

In [18]:
vis_util.visualize_boxes_and_labels_on_image_array( 
    image, 
    np.squeeze(boxes), 
    np.squeeze(classes).astype(np.int32), 
    np.squeeze(scores), 
    category_index, 
    use_normalized_coordinates = True, 
    line_thickness = 2, 
    min_score_thresh = 0.70) 

array([[[ 28,  39,  36],
        [ 30,  41,  38],
        [ 33,  44,  41],
        ...,
        [ 49,  47,  46],
        [ 44,  42,  41],
        [ 51,  49,  48]],

       [[ 23,  34,  31],
        [ 25,  36,  33],
        [ 27,  38,  35],
        ...,
        [ 36,  34,  33],
        [ 46,  44,  43],
        [ 46,  44,  43]],

       [[ 22,  33,  30],
        [ 23,  34,  31],
        [ 25,  36,  33],
        ...,
        [ 27,  25,  24],
        [ 42,  40,  39],
        [ 48,  46,  45]],

       ...,

       [[ 93,  74,  71],
        [102,  85,  82],
        [128, 111, 108],
        ...,
        [123, 115, 115],
        [183, 175, 175],
        [122, 117, 116]],

       [[ 86,  67,  64],
        [ 85,  68,  65],
        [111,  94,  91],
        ...,
        [107,  99,  99],
        [169, 161, 161],
        [112, 107, 106]],

       [[ 78,  59,  56],
        [ 74,  57,  54],
        [ 84,  67,  64],
        ...,
        [ 89,  81,  81],
        [159, 151, 151],
        [112, 107, 106]]

In [19]:
cv2.imshow('Object detector', image)
cv2.imwrite('objectdetection123.jpg', image) 

True

In [20]:
cv2.waitKey(0) 

-1

In [21]:
cv2.destroyAllWindows()
